Three main methods of getting data into a Tensorflow program like below:
* Feeding: Python code provides the data when running each step;
* Reading from files: an input pipeline reads the data from files at the beginning of a Tensorflow graph;
* Preloaded data: a constant or variable in the Tensorflow graph holds all the data(for small datasets).

Feeding:
Supply feed data through the feed_dict argument to a run() or eval() call that initiates computation.
We can replace any Tensor with feed data,including variables and constants, the best practice is touse a tf.placeholder node. A placeholder exists solely to serve as the target of feeds. It's not initialized and contains no data.And a placeholder will generate an error if it's executed without a feed.

Reading from files:
A typical pipeline for reading records from files has the following stages:
1. The list of filenames;
2. Optional filename shuffling;
3. Optional epoch limit;
4. Filename queue;
5. A Reader for the file format;
6. A decoder for a record read by the reader;
7. Optional preprocessing;
8. Example queue.

Filenames, shuffling, and epoch limits:
List of filename like a constant string Tensor ["file0", "file1"] or [("file%d" % i) for i in xrange(10)] or we could use tf.train.match_filename_once function.

Pass the filename list to the tf.train.string_input_producer function, and it creates a FIFO queue for holding the filenames until the reader needs them. It has options for shuffling and setting a maximum number of epochs. A queue runner adds the whole list of filenames to the queue once for each epoch, shuffling the filenames within an epoch if shuffle = True.

The queue runner works in a thread separate from the reader that pulls filenames from the queue, so te shuffling and enqueuing process does not block the reader.

File formats:
Select the reader that matches our input file format and pass the filename queue to the reader's read method. The read method outputs a key identifying the file and record(useful for debugging if you have some weird records), and a scalar string value. Use one of the decoder and conversion ops to decode this string into the tensors that make up an example.
csv files -> tf.TextLineReader (with tf.decode_csv op to decode the string returned).
binary files -> tf.FixedLengthRecordReader ( with tf.decode_raw op...)
Also we can convert whatever data we have into a supported format. And it makes it easier to mix and match data sets and network architectures. The recommended format is a TFRecord file containing tf.train.Example protocol buffers(which contain Features as a field). We can write a little program that get our data,stuffs it in an Example protocol buffer, serializes the protocol buffer to a string, and then writes the string to a TFRecords file using the tf.python_io.TFRecordWriter. (This mechanism is very like caffe's input file processing method which translate data files into lmdb or leveldb database files).

To read a file of TFRecords, use tf.TFRecordReader with the tf.parse_single_example decoder.

We must call tf.train.start_queue_runners to populate the queue before we can run or eval to execute the reader's read op. Otherwise it will block while waits for filenames from the queue.

Preprocessing:
We can then do any preprocessing on these examples we want. This would be any processing that doesn't depend on trainable parameters. Actions like normalization of our data, picking a random slice, adding noise or distortions.

Batching:
At the end of the pipeline we use another queue to batch together examples for training, evaluation, or inference. For this we use a queue that randomizes the order of examples, using the tf.train.shuffle_batch.

In [ ]:
#Below is an example for the whole data input pipeline
def read_my_file_format(filename_queue):
    reader = tf.SomeReader()
    key, record_string = reader.read(file_name_queue)
    example, label = tf.some_decoder(record_string)
    processed_example = some_processing(example)
    return processed_example, label

def input_pipeline(filenames, batch_size, num_epochs = None):
    filename_queue = tf.train.string_input_producer(filenames, num_epochs = num_epochs, shuffle = True)
    example, lable = read_my_file_format(filename_queue)
    # min_after_dequeue defines how big a buffer we will randomly sample from -- bigger means better shuffling 
    # but slower start up and more memory used.
    # capacity must be larger than min_after_dequeue and the amount larger determines the maximum we will prefetch.
    min_after_dequeue = 10000
    capacity = min_after_dequeue + 3 * batch_size
    # If we need more parallelism or shuffling of examples between files, use multiple reader instances using the
    # tf.train.shuffle_batch_join
    example_bath,label_batch = tf.train.shuffle_batch(
        [example, label], batch_size, capacity = capacity,
        min_after_dequeue = min_after_dequeue)

def input_pipeline_parallel(filenames, batch_size, read_threads, num_epochs = None):
    filename_queue = tf.train.string_input_producer(
        filenames, num_epochs = num_epochs, shuffle = True)
    example_list - [read_my_file_format(filename_queue) for _ in xrange(read_threads)]
    min_after_dequeue = 10000
    capacity = min_after_dequeue + 3 * batch_size
    example_bath, label_batch = tf.train.shuffle_batch_join(
        example_list, batch_size = batch_size,capacity = capacity,
        min_after_dequeue = min_after_dequeue)
    return example_bath, label_batch

Creating threads to prefetch using QueueRunnder objects:
Many of the tf.train functions listed above add tf.train.QueueRunner objects to our graph, this requires us call tf.train.start_queue_runners before running any training or inference op or it will hang forever.
So once the graph is constructed, the tf.train.start_queue_runners function asks each QueueRunner in the graph to start its threads running the enqueuing operations.

Notes:
SparseTensors don't play well with queues. If we use SparseTensors we have to decode the string records using tf.parse_example after batching.

Preloaded data:
Only used for small data sets that can be loaded entirely in memory. Two approaches like below:
a. Store the data in a constant(simpler, but uses more memory, since the constant is stored inline in the graph data structure, which may be duplicated a few times);
b. Store the data in a variable, that you initialize and then never change(need to initialize it after the graph has been built)(setting trainable = False keeps the variables out of the GraphKeys.TRAINABLE_VARIABLES collection in the graph. Setting collections = [] keeps the variable out of the GraphKeys.GLOBAL_VARIABLES collection used for saving and restoring checkpoints.).

Multiple input pipelines:
Commonly we will want to train on one dataset and evaluate on another. One way to do this is to actually have two separate processes:
* The training process reads training input data and periodically writes checkpoint files with all the trained variables;
* The evaluation process restores the checkpoint files into an inference model that reads validation input data.
Also we can have the train and eval in the same graph in the same process, and share their trained variables.